[ 동적 모델 설계 구현 ]

- 사용 모듈 : nn.ModuleList
- 특징 : 일반 list로는 pytorch에서 layer 인식 안됨! -> 대안 : ModuleList

In [ ]:
# 모델 설계
# 입력층
# 은닉층 : 유동적 
# 출력층 

In [2]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
from torch.utils.data import Dataset, DataLoader                ## 데이터셋 관련 
import torch.optim as optim                                     ## 최적화 모듈
from torchmetrics.classification import *                       ## 모델 성능지표 관련
from torchinfo import summary                                   ## 모델 구조 및 정보 관련 

from sklearn.model_selection import train_test_split            ## 학습용 & 테스트용 데이터셋 분리
from sklearn.preprocessing import MinMaxScaler, StandardScaler  ## 연속형 피쳐 스케일링
from sklearn.preprocessing import OneHotEncoder, LabelEncoder   ## 범주형 피쳐, 타겟 인코딩


In [ ]:
# [1] 은닉층의 갯수가 동적인 모델 

class MyModel(nn.Module):
    def __init__(self, in_in, out_out, h_in, h_cnt):    # h_cnt : 은닉층의 갯수  
        # 부모 클래스 생성
        super().__init__()
        
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(in_in, h_in)
        self.h1_layer = nn.ModuleList([nn.Linear(h_in,h_in) for _ in range(h_cnt)])
        self.output_layer = nn.Linear(h_in, out_out)
        
     # 순전파학습메서드------------------------------------------------
    def forward(self, x):
        y = F.relu(self.input_layer(x))
        
        for linear in self.h1_layer:
            y = F.relu(linear(y))
            
        return self.output_layer(y)

In [11]:
# 모델 인스턴스 생성 및 테스트
# in_in = 3
# out_out = 1
# hd_in = 5
# hd_cnt = 1

m1 = MyModel(3,1,5,1)

m1(torch.FloatTensor([[1,2,3]]))

summary(m1,input_size=(100,3))

Layer (type:depth-idx)                   Output Shape              Param #
MyModel                                  [100, 1]                  --
├─Linear: 1-1                            [100, 5]                  20
├─ModuleList: 1-2                        --                        --
│    └─Linear: 2-1                       [100, 5]                  30
├─Linear: 1-3                            [100, 1]                  6
Total params: 56
Trainable params: 56
Non-trainable params: 0
Total mult-adds (M): 0.01
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

In [10]:
# 모델 인스턴스 생성 및 테스트
# in_in = 3
# out_out = 1
# hd_in = 5
# hd_cnt = 1

m2 = MyModel(3,3,10,5)

m2(torch.FloatTensor([[1,2,3]]))

tensor([[-0.2294,  0.1871,  0.1614]], grad_fn=<AddmmBackward0>)

In [ ]:
# [2] 은닉층 갯수 및 뉴런 갯수 동적인 모델

class DynamicModel(nn.Module):
    def __init__(self, in_in, in_out, out_out, h_outs=[]):    # in_in : 입력층의 입력 / out_out : 출력층의 출력 
        # 부모 클래스 생성                                        3  10  3  7  5
        super().__init__()
        
        # 입력층 생성
        self.input_layer = nn.Linear(in_in, in_out)
        
        # 은닉층 여러개 생성 
        self.h1_layer = nn.ModuleList()
        for idx in range(len(h_outs)):
            h_in = h_outs[idx-1] if idx else in_out 
            h_out = h_outs[idx]
            self.h1_layer.append(nn.Linear(h_in, h_out)) 
        
        # 출력층 생성
        self.output_layer = nn.Linear(h_outs[-1] if len(h_outs) else in_out, out_out)
        
     # 순전파학습메서드------------------------------------------------
    def forward(self, x):
        y = F.relu(self.input_layer(x))
        
        for linear in self.h1_layer:
            y = F.relu(linear(y))
            
        return self.output_layer(y)

In [ ]:
# 모델 인스턴스 생성 및 테스트
# h_out 할당 X 버전

m1 = DynamicModel(3,5,1)

m1(torch.FloatTensor([[1,2,3]]))

summary(m1,input_size=(100,3))

Layer (type:depth-idx)                   Output Shape              Param #
DynamicModel                             [100, 1]                  --
├─Linear: 1-1                            [100, 5]                  20
├─Linear: 1-2                            [100, 1]                  6
Total params: 26
Trainable params: 26
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

In [16]:
# 모델 인스턴스 생성 및 테스트
# h_out 할당 O 버전

m2 = DynamicModel(3,10,3,[7,5,4,3,2])

m2(torch.FloatTensor([[1,2,3]]))

summary(m2,input_size=(100,3))

Layer (type:depth-idx)                   Output Shape              Param #
DynamicModel                             [100, 3]                  --
├─Linear: 1-1                            [100, 10]                 40
├─ModuleList: 1-2                        --                        --
│    └─Linear: 2-1                       [100, 7]                  77
│    └─Linear: 2-2                       [100, 5]                  40
│    └─Linear: 2-3                       [100, 4]                  24
│    └─Linear: 2-4                       [100, 3]                  15
│    └─Linear: 2-5                       [100, 2]                  8
├─Linear: 1-3                            [100, 3]                  9
Total params: 213
Trainable params: 213
Non-trainable params: 0
Total mult-adds (M): 0.02
Input size (MB): 0.00
Forward/backward pass size (MB): 0.03
Params size (MB): 0.00
Estimated Total Size (MB): 0.03